In [1]:
import pandas as pd
import numpy as np
from feature_engineering.bureau_balance_fe import bureau_balance
from feature_engineering.bureau_fe import bureau
from feature_engineering.application_fe import application
from feature_engineering.credit_card_balance_fe import credit_card_balance
from feature_engineering.installment_payment_fe import installments_payments
from feature_engineering.pos_cash_balance_fe import pos_cash_balance
from feature_engineering.previous_application_fe import previous_application
from model import train_logistic_regression, train_decision_tree
from sklearn.preprocessing import StandardScaler
from merge import merge_all, process_and_select_features
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

Done!


In [2]:
feature_app_test = application(pd.read_csv(r"C:\Users\Admin\Desktop\Final data prep\cleaned_data\application_test.csv"))
feature_app_train = application(pd.read_csv(r"C:\Users\Admin\Desktop\Final data prep\cleaned_data\application_train.csv"))
feature_bureau_balance = bureau_balance(pd.read_csv(r"C:\Users\Admin\Desktop\Final data prep\cleaned_data\bureau_balance.csv"))
feature_bureau = bureau(pd.read_csv(r"C:\Users\Admin\Desktop\Final data prep\cleaned_data\bureau.csv"),feature_bureau_balance)
feature_prev = previous_application(pd.read_csv(r"C:\Users\Admin\Desktop\Final data prep\cleaned_data\prev_application.csv"))
feature_pos = pos_cash_balance(pd.read_csv(r"C:\Users\Admin\Desktop\Final data prep\cleaned_data\POS_CASH_balance.csv"))
feature_credit = credit_card_balance(pd.read_csv(r"C:\Users\Admin\Desktop\Final data prep\cleaned_data\credit_card_balance.csv"))
feature_install = installments_payments(pd.read_csv(r"C:\Users\Admin\Desktop\Final data prep\cleaned_data\installment_payment.csv"))

c:\Users\Admin\Desktop\Final data prep\feature_engineering\application_fe.py:33: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  application_encoded['INCOME_PER_PERSON'] = (
c:\Users\Admin\Desktop\Final data prep\feature_engineering\application_fe.py:37: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  application_encoded['ANNUITY_INCOME_RATIO'] = (
c:\Users\Admin\Desktop\Final data prep\feature_engineering\application_fe.py:41: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` 

In [4]:
merge_app_train = merge_all(feature_app_train, feature_bureau, feature_prev, feature_pos, feature_credit, feature_install)
merge_app_test = merge_all(feature_app_test, feature_bureau, feature_prev, feature_pos, feature_credit, feature_install)

After merging bureau: (246009, 437)
After merging previous application: (246009, 677)
After merging POS CASH balance: (246009, 702)
After merging credit card balance: (246009, 738)
After merging installments payments: (246009, 761)
Filling missing values...
After filling missing values:
TARGET                                                                   0
CNT_CHILDREN                                                             0
AMT_INCOME_TOTAL                                                         0
AMT_CREDIT                                                               0
AMT_ANNUITY                                                              0
AMT_GOODS_PRICE                                                          0
REGION_POPULATION_RELATIVE                                               0
DAYS_BIRTH                                                               0
DAYS_EMPLOYED                                                            0
DAYS_REGISTRATION                    

In [5]:
X_train = merge_app_train.drop('TARGET', axis=1)
y_train = merge_app_train['TARGET']
X_test = merge_app_test

In [ ]:
# Assuming get_important_features is implemented
X_train_reduced, X_test_reduced, y_train = process_and_select_features(X_train, y_train, X_test=X_test, threshold=0.0001)

In [ ]:
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from imblearn.over_sampling import SMOTE
from sklearn.impute import SimpleImputer

# Assuming you have already loaded your data and X_train, y_train are defined

# Handling missing values
imputer = SimpleImputer(strategy='median')
X_train_imputed = imputer.fit_transform(X_train_reduced)

# Scaling data
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train_imputed)

# Apply SMOTE for class imbalance
smote = SMOTE(random_state=42)
X_train_resampled, y_train_resampled = smote.fit_resample(X_train_scaled, y_train)

# Check the class distribution before and after SMOTE
print("Before SMOTE, class distribution:", y_train.value_counts())
print("After SMOTE, class distribution:", y_train_resampled.value_counts())


Before SMOTE, class distribution: TARGET
0.0    226133
1.0     19876
Name: count, dtype: int64
After SMOTE, class distribution: TARGET
0.0    226133
1.0    226133
Name: count, dtype: int64


In [8]:
lr_model, lr_params, lr_pred_proba = train_logistic_regression(
    X_train_scaled, y_train, 
    X_train_scaled, y_train,  
    param_file='best_params_logistic.pkl',
    max_evals=5  # Optional: adjust number of evaluations
)

  0%|          | 0/5 [00:00<?, ?trial/s, best loss=?]

INFO:hyperopt.tpe:build_posterior_wrapper took 0.000000 seconds
INFO:hyperopt.tpe:TPE using 0 trials


 20%|██        | 1/5 [06:26<25:47, 386.88s/trial, best loss: -0.7742766805402428]

INFO:hyperopt.tpe:build_posterior_wrapper took 0.009231 seconds
INFO:hyperopt.tpe:TPE using 1/1 trials with best loss -0.774277


 40%|████      | 2/5 [1:35:30<2:45:07, 3302.54s/trial, best loss: -0.7742766805402428]

...

 60%|██████    | 3/5 [1:40:16<1:04:10, 1925.01s/trial, best loss: -0.7742766805402428]

INFO:hyperopt.tpe:build_posterior_wrapper took 0.004011 seconds
INFO:hyperopt.tpe:TPE using 3/3 trials with best loss -0.774277


Error during evaluation: Invalid combination: {'C': 0.05636577469429527, 'class_weight': None, 'dual': True, 'l1_ratio': 0.060502851709994254, 'max_iter': 600.0, 'penalty': 'l2', 'solver': 'saga'}, Params: {'C': 0.05636577469429527, 'class_weight': None, 'dual': True, 'l1_ratio': 0.060502851709994254, 'max_iter': 600.0, 'penalty': 'l2', 'solver': 'saga'}
 60%|██████    | 3/5 [1:40:16<1:04:10, 1925.01s/trial, best loss: -0.7742766805402428]

INFO:hyperopt.tpe:build_posterior_wrapper took 0.001116 seconds
INFO:hyperopt.tpe:TPE using 4/4 trials with best loss -0.774277


Error during evaluation: Invalid combination: {'C': 0.00017459615680667066, 'class_weight': 'balanced', 'dual': False, 'l1_ratio': 0.013995762035543535, 'max_iter': 800.0, 'penalty': 'elasticnet', 'solver': 'lbfgs'}, Params: {'C': 0.00017459615680667066, 'class_weight': 'balanced', 'dual': False, 'l1_ratio': 0.013995762035543535, 'max_iter': 800.0, 'penalty': 'elasticnet', 'solver': 'lbfgs'}
100%|██████████| 5/5 [1:40:16<00:00, 1203.23s/trial, best loss: -0.7742766805402428]
Saving best parameters to best_params_logistic.pkl
Parameters saved successfully!


0.7728

In [9]:
#dt_model, dt_params, dt_pred_proba = train_decision_tree(
#    X_train_scaled, y_train, 
#    X_train_scaled, y_train,  
#    param_file='best_params_logistic.pkl',
#    max_evals=200  # Optional: adjust number of evaluations
#)

In [10]:
# Step 5: Dự đoán trên tập kiểm tra Kaggle (X_test từ Kaggle)
# Tiền xử lý tập X_test
X_test_imputed = imputer.transform(X_test_reduced)
X_test_scaled = scaler.transform(X_test_imputed)

In [11]:
lr_predictions = lr_model.predict_proba(X_test_scaled)[:, 1]
#dt_predictions = dt_model.predict_proba(X_test_scaled)[:, 1]

In [12]:
# Tạo file submission
submission_lr = pd.DataFrame({
    'SK_ID_CURR': X_test['SK_ID_CURR'],
    'Target': lr_predictions
})
submission_lr['SK_ID_CURR'] = submission_lr['SK_ID_CURR'].astype(int)
submission_lr.to_csv('submission_logistic_regression.csv', index=False)

#submission_dt = pd.DataFrame({
#    'SK_ID_CURR': X_test['SK_ID_CURR'],
#    'Target': dt_predictions
#})
#submission_dt['SK_ID_CURR'] = submission_dt['SK_ID_CURR'].astype(int)
#submission_dt.to_csv('submission_decision_tree.csv', index=False)